In [16]:
import os
import numpy as np
import pandas as pd
from nltk.tag.stanford import CoreNLPPOSTagger, CoreNLPNERTagger
from nltk.parse.corenlp import CoreNLPParser

stpos, stner = CoreNLPPOSTagger(), CoreNLPNERTagger()
from nltk.tokenize import sent_tokenize

In [17]:
import pickle

# How to use : create a folder name "repo" in the directory
# where file is located
# obj : the object required to save
# name : the name for the host file of the data

def saveObject(obj,name): 
    pickle.dump(obj,open( "repo/"+name+".pkl", "wb" ))

def loadObject(name):
    obj = pickle.load( open( "repo/"+name+".pkl", "rb" ) )
    return obj

## List of Syntactic Feature and which Feature will come from where

1. Mean length of clause (MLC)
2. Mean length of a sentence (MLS)
- Mean length of T-unit (MLT)
- Num. of Clauses per Sentence (C/S)
- Num. of T-Units per sentence (T/S)
- Num. of Clauses per T-unit (C/T)
- Num. of Complex-T-Units per T-unit (CT/T)
- Dependent Clause to Clause Ratio (DC/C)
- Dependent Clause to T-unit Ratio (DC/T)
- Co-ordinate Phrases per Clause (CP/C)
- Co-ordinate Phrases per T-unit (CP/T)
- Complex Nominals per Clause (CN/C)
- Complex Nominals per T-unit (CN/T)
- Verb phrases per T-unit (VP/T)
### All the above 14 will be calculated from the library. 
#### Rest.
- Avg. Parse Tree Height ## Code done
- Verb phrase per Sentense ##VP/S in Lib result
- Noun Phrase per Sentense #################Need to be implemented
- PP per Sentence ###############Need to be implement
- Avg len of ###############3NP, PP, VP need to be implement
- Num. Dependent Clauses per sentence (NumDC) #LIB
- Num. Complex-T units per sentence (NumCT) #LIB
- Num. Co-ordinate Phrases per sentence (CoOrd) #LIB
- Num. SBARs per sentence (NumSBAR) ## Code Done

In [18]:
#Please Execute the following command in the new terminal to switch on stanford Parsar

#1. #cd /home/sp/stanford-corenlp-full-2016-10-31/
#2. #java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -preload tokenize,ssplit,pos,lemma,parse,depparse -status_port 9000 -port 9000 -timeout 15000



#For Stanford NLP Parsar
parser = CoreNLPParser(url='http://localhost:9000')

In [19]:
############ Not Required Anymore ##############
# def noOfClause(sent):
#     trre = parser.raw_parse(sent)
#     for t2 in trre:
#         t2
#     labAr = np.array([],dtype=str)
#     for ll in t2.subtrees():
#         labAr = np.append(labAr,ll.label())
#     return sum(labAr == "S")
###################################################

# Read the all the files present in a directory
def dataReader(flistName,location):
    dataList = []
    for fname in flistName:
        file = open(location+fname,"r",encoding="utf-8")
        data = file.read()
        dataList.append(data)
    return dataList

def levelWordDoc(dataList,nameList):
    featureDict = {}
    L1 = {}
    L2 = {}
    L3 = {}
    L4 = {}
    L5 = {}
    L6 = {}
    i = 0
    
    for page in dataList:
        l1 = 0
        l2 = 0
        l3 = 0
        l4 = 0
        l5 = 0
        l6 = 0
        
        wordAsTokensList = nltk.work_tokenize(page)
        for ww in wordAsTokensList:
            if ww in level1Set:
                l1 += 1
            elif ww in level2Set:
                l2 += 1
            elif ww in level3Set:
                l3 += 1
            elif ww in level4Set:
                l4 += 1
            elif ww in level5Set:
                l5 += 1
            elif ww in level6Set:
                l6 += 1
        key = nameList[i]
        L1[key] = l1
        L2[key] = l2
        L3[key] = l3
        L4[key] = l4
        L5[key] = l5
        L6[key] = l6
    
    featureDict["L1"] = L1
    featureDict["L2"] = L2
    featureDict["L3"] = L3
    featureDict["L4"] = L4
    featureDict["L5"] = L5
    featureDict["L6"] = L6
    
    return featureDict

    
        

############ Important #################################
# Return a distionary, Senctence Count, VP_S, NP_S, PP_S, Avg_VP, Avg_PP, Avg_NP, Sbar_S, treeH_S
def FeatureCalulator(dataList,nameList):
    featureDict = {}
    VP_S = {}
    NP_S = {}
    PP_S = {}
    Avg_VP = {}
    Avg_NP = {}
    Avg_PP = {}
    Sbar_S = {}
    treeH_S = {}
    Total_S = {}
    i = 0
    for page in dataList:
        
        VP = 0     # No. of Verb Phrase in a Page
        VPLen = 0  # Len of All the Verb Phrase in a Page
        NP = 0
        NPLen = 0
        PP = 0
        PPLen = 0
        Sen = 0
        Sbar = 0
        TreeH = 0
        
        
        # Deleting Last Line since doesn't contain any text
        #del paras[len(paras)-1] 
        
        sentences = sent_tokenize(page)
        for sentence in sentences:
            if sentence != "":
                #Core Feature Calculator
                Sen = Sen + 1
                ttpp = parser.raw_parse(sentence)
                parsedText = next(ttpp)

                (locH,locSbarC) = getHofPTreeAndSbarCount(parsedText)
                Sbar += locSbarC
                TreeH += locH

                (locVPCount,locVPLen) = getVPLenAndCount(parsedText)
                VP += locVPCount
                VPLen += locVPLen

                (locNPCount,locNPLen) = getNPLenAndCount(parsedText)
                NP += locNPCount
                NPLen += locNPLen

                (locPPCount,locPPLen) = getPPLenAndCount(parsedText)
                PP += locPPCount
                PPLen += locPPLen

         ########## The Conclusion ########################
        key = nameList[i]
        i += 1
        print(i)
        if VP == 0:
            VP = 1
        if NP == 0:
            NP = 1
        if PP == 0:
            PP = 1
        if Sen == 0:
            Sen = 1
        VP_S[key] = VP/Sen
        NP_S[key] = NP/Sen
        PP_S[key] = PP/Sen
        Avg_VP[key] = VPLen/VP
        Avg_NP[key] = NPLen/NP
        Avg_PP[key] = PPLen/PP
        Sbar_S[key] = Sbar/Sen
        treeH_S[key] = TreeH/Sen
        Total_S[key] = Sen
        
    featureDict["VP_S"] = VP_S
    featureDict["NP_S"] = NP_S
    featureDict["PP_S"] = PP_S
    featureDict["Avg_NP"] = Avg_NP
    featureDict["Avg_VP"] = Avg_VP
    featureDict["Avg_PP"] = Avg_PP
    featureDict["Sbar_S"] = Sbar_S
    featureDict["treeH_S"] = treeH_S
    featureDict["Total_S"] = Total_S
    
    return featureDict
##################################################################            

def getHofPTreeAndSbarCount(treeParsed):
    #treeParsed = next(parsedText)
    height_tree = treeParsed.height()
    sbarCount = 0
    for subtrees in treeParsed.subtrees():
        if subtrees.label() == 'SBAR':
            sbarCount = sbarCount + 1
    return (height_tree,sbarCount)

#input : Sentence. Output (No of VP, Total Length of VP)
def getVPLenAndCount(textTree):
    #parsedText = parser.raw_parse(sentence)
    #textTree = next(parsedText)
    Count = 0
    LenCount = 0
    for subtree in textTree.subtrees():
        if subtree.label() == 'VP':
            Count = Count + 1
            LenCount += len(subtree.leaves())
    return (Count,LenCount)

#input : Sentence. Output (No of NP, Total Length of NP)
def getNPLenAndCount(textTree):
    #parsedText = parser.raw_parse(sentence)
    #textTree = next(parsedText)
    Count = 0
    LenCount = 0
    for subtree in textTree.subtrees():
        if subtree.label() == 'NP':
            Count = Count + 1
            LenCount += len(subtree.leaves())
    return (Count,LenCount)

#input : Sentence. Output (No of PP, Total Length of PP)
def getPPLenAndCount(textTree):
    #parsedText = parser.raw_parse(sentence)
    #textTree = next(parsedText)
    Count = 0
    LenCount = 0
    for subtree in textTree.subtrees():
        if subtree.label() == 'PP':
            Count = Count + 1
            LenCount += len(subtree.leaves())
    return (Count,LenCount)



In [20]:
#Read the Data
loc = "/home/sp/nlpfinal/NCERT_Data/extracted_data/textData/"
#BitGCSE, BitKS3, WRLevel2, WRLevel3, WRLevel4

class1 = loc + "WRLevel2/"
class2 = loc +  "WRLevel3/"
class3 = loc +  "WRLevel4/"
class4 = loc +  "BitKS3/"
class5 = loc +  "BitGCSE/"

c1 = os.listdir(class1)
c2 = os.listdir(class2)
c3 = os.listdir(class3)
c4 = os.listdir(class4)
c5 = os.listdir(class5)

l1Data = dataReader(c1,class1)
l2Data = dataReader(c2,class2)
l3Data = dataReader(c3,class3)
l4Data = dataReader(c4,class4)
l5Data = dataReader(c5,class5)


In [25]:
FeatureDictL1 = FeatureCalulator(l1Data,c1)
print("L1 Done")
FeatureDictL2 = FeatureCalulator(l2Data,c2)
print("L2 Done")
FeatureDictL3 = FeatureCalulator(l3Data,c3)
print("L3 Done")
FeatureDictL4 = FeatureCalulator(l4Data,c4)
print("L4 Done")
FeatureDictL5 = FeatureCalulator(l5Data,c5)
print("L5 Done")


1
2
3
4
5
6


ReadTimeout: HTTPConnectionPool(host='localhost', port=9000): Read timed out. (read timeout=60)

In [114]:
# saveObject(FeatureDictL1,"featL1")
# saveObject(FeatureDictL2,"featL2")
# saveObject(FeatureDictL3,"featL3")
#L3 Done

In [7]:
#saveObject(FeatureDictL4,"featL4")
#saveObject(FeatureDictL5,"featL5")

In [11]:
FeatureDictL4

NameError: name 'FeatureDictL4' is not defined

In [24]:
c1

['aeen101.txt',
 'aeen110.txt',
 'aeen107.txt',
 'aeen104.txt',
 'aeen102.txt',
 'aeen103.txt',
 'aeen110.txt.parsed',
 'aeen106.txt',
 'aeen105.txt',
 'aeen109.txt',
 'aeen108.txt']